### 悬崖漫步问题：
1. 要求一个智能体从起点出发，避开悬崖行走，最终到达目标位置。有一个 4×12 的网格世界，每一个网格表示一个状态。
2. 智能体的起点是左下角的状态，目标是右下角的状态，智能体在每一个状态都可以采取 4 种动作：上、下、左、右。
3. 如果智能体采取动作后触碰到边界墙壁则状态不发生改变，否则就会相应到达下一个状态。
4. 环境中有一段悬崖，智能体掉入悬崖或到达目标状态都会结束动作并回到起点，也就是说掉入悬崖或者达到目标状态是终止状态。
5. 智能体每走一步的奖励是 −1，掉入悬崖的奖励是 −100。

#### ReplayBuffer类：支撑off-policy

In [7]:
import random
import gym
import numpy as np
import abc # 用于定义抽象类和抽象方法的模块。

In [ ]:
class ReplayBuffer():
    def __init__(self, max_size = 80):
        # 存储状态对的总量
        self.max_size = max_size
        self.buffer = []
    def push_transition(self, transition):
        if transition not in self.buffer: # 这是遍历做法，适合小样本
            self.buffer.append(transition)
            if len(self.buffer) > self.max_size:
                self.buffer = self.buffer[-self.max_size:] # 只保留最新的max_size个状态对(只去掉第一个)
                
    def sample(self, batch_size = 5):
        # 判定batch_size是否大于buffer长度
        if batch_size > self.max_size:
            raise ValueError("采样的长度大于经验回放池大小！")
        # 随机采样
        return random.sample(self.buffer, min(batch_size, len(self.buffer)))
    def isfull(self):
        """某些强化学习算法（如 off-policy Q-learning）会等 buffer 满了才开始更新；"""
        return len(self.buffer) == self.max_size

#### 对于各种 TD Learning 方法而言，除了更新Q估计的具体操作不同外，其他几乎都相同。故抽象出基类class Solver

强化学习中时序差分（TD）控制算法的抽象基类。

本类提供了基于值函数的控制方法（如 **Q-learning**、**SARSA**、**Expected SARSA**）的通用框架结构，  
封装了 ε-greedy 策略、Q 表初始化、贪婪策略更新、状态值计算等通用功能。

子类需实现 `update_Q_table()` 方法，以定义具体的值函数更新规则。

---

##### 参数说明

| 参数名 | 类型 | 说明 |
|--------|------|------|
| `env` | `gym.Env` | Gym 环境实例，需具有 **离散的 observation 和 action 空间** |
| `alpha` | `float` | 学习率，控制 Q 值更新的幅度 |
| `gamma` | `float` | 折扣因子，用于对未来奖励进行折现 |
| `epsilon` | `float` | ε-greedy 策略中的探索概率 |
| `seed` | `int or None` | 随机种子，确保探索策略可复现 |
| `replay_buffer_size` | `int` | 经验回放池最大容量，支持 off-policy 算法 |

---

##### 属性说明
| 属性名 | 类型 | 说明 |
|--------|------|------|
| `Q_table` | `np.ndarray` | 状态-动作值函数 Q(s, a) |
| `V_table` | `np.ndarray` | 状态值函数 V(s)，仅用于可视化 |
| `greedy_policy` | `List[np.ndarray]` | 每个状态下的最优动作集合（支持多个并列最优） |
| `policy_is_updated` | `bool` | 策略是否已根据最新 Q 表更新 |
| `rng` | `np.random.RandomState` | 控制探索行为的随机数生成器 |
| `replay_buffer` | `ReplayBuffer` | 经验回放缓存器，用于支持经验采样 |

---
##### 注意事项

- 本类为 **抽象基类**（使用 `abc` 模块），不能直接实例化；
- 必须由子类实现 `update_Q_table()` 方法；
- 仅支持 **离散状态空间和动作空间**；
- 策略提升使用 `greedy_policy`，支持多最优动作随机选择；
- 与 `ReplayBuffer` 配合可实现 off-policy 学习结构（如 DQN、Q-learning）。


In [ ]:
class Solver():
    def __init__(self,env:gym.Env, alpha=0.1, gamma=0.9, epsilon=0.1, seed=None, replay_buffer_size=80):
        # 初始化函数
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        # 环境中提取动作空间大小和状态空间大小（要求 env.action_space 和 env.observation_space 必须是 gym.spaces.Discrete 类型）
        self.n_action = env.action_space.n
        self.n_state = env.observation_space.n
        # 初始化Q值表,每个状态-动作对的初始值设为 0（代表“完全不了解环境”）
        self.Q_table = np.zeros((self.n_state, self.n_action),dtype = np.float32)
        # 初始化当前策略的状态值
        self.V_table = np.zeros((self.n_state),dtype = np.float32)
        # 初始时默认每个动作都是最优的(每个状态对应的最优动作列表，而这个列表的长度是不确定的、可能不同的->object)
        self.greedy_policy = np.array([np.arange(self.n_action)] * self.n_state,dtype = object)
        # 标志变量：表示当前 greedy_policy 是否与最新 Q 表匹配(当 Q 值更新后，应将其设为 False)
        self.policy_is_updated = False
        self.rng = np.random.RandomState(1) # 每个智能体独立随机数，不影响全局
        # 设置经验回放池
        self.replay_buffer = ReplayBuffer(replay_buffer_size)
    
    def take_action(self, state):
        """用于epsilon-greedy策略选择动作,该部分属于policy improvement的范畴"""
        # 确保策略已经更新
        if not self.policy_is_updated:
            self.update_policy()
        # epsilon-greedy 策略选择动作
        if np.random.rand() < self.epsilon:
            return self.rng.randint(self.n_action) # 随机选择动作(从 [0, n) 中选一个整数)
        else:
            return self.rng.choice(self.greedy_policy[state]) # 从当前最优动作中随机选择一个，鼓励策略多样性
    
    def update_policy(self):
        """更新当前策略,从Q_table中提取最优动作"""
        # 找出所有最大的Q值对应的动作
        # 这里的np.where()返回的是一个元组，元组中包含了所有最大值的索引[0]将ndarray提取出来
        self.greedy_policy = np.array([np.where(self.Q_table[s] == np.max(self.Q_table[s]))[0] 
                                        for s in range(self.n_state)], dtype=object)
        # 策略更新标志设为 True
        self.policy_is_updated = True
    
    def update_V_table(self):
        """根据当前 Q 表和贪婪策略计算每个状态的状态值函数 V(s)
        若某个状态是接近终点或高奖励区域，那么它的状态值函数 V(s) 会较高；
        若某个状态是接近障碍物或低奖励区域，那么它的状态值函数 V(s) 会较低；
        如果 V 值从左到右、从起点向终点逐渐升高，说明策略在学习从起点走向目标；"""
        # 判断策略是否更新
        if not self.policy_is_updated:
            self.update_policy()
        # 计算每个状态对的状态函数(V(s)=max_a Q(s,a)=E_(a~pi(·|s))[Q(s,a)])
        for s in range(self.n_state):
            self.V_table[s] = self.Q_table[s][self.greedy_policy[s][0]]
            
    @abc.abstractmethod
    def update_Q_table(self):
        """抽象实现"""
        pass

#### 一、SARSA：
1. Sarsa为一种on-policy算法，使用当前策略π来于环境交互。在每轮交互时先采样得到transition $(s_t,a_t,r_t,s_{t+1},a_{t+1})$。
2. **Bellman Expectation Equation**进行更新，并对当前策略进行评估：
$$
Q_\pi(s_t, a_t) \leftarrow Q_\pi(s_t, a_t) + \alpha \Bigl( r + \gamma Q_\pi(s_{t+1}, a_{t+1}) - Q_\pi(s_t, a_t) \Bigr)
$$
其中：
- $\alpha$：学习率
- $\gamma$：折扣因子
- $Q_\pi(s,a)$：当前策略下的状态-动作值
- $a_{t+1} \sim \pi(\cdot | s_{t+1})$：下一步动作依旧从当前策略中采样（如 ε-greedy）
3. 再使用贪婪算法选取某个状态下动作价值最大的那个动作。

In [10]:
class Sarsa(Solver):
    def __init__(self, env:gym.Env, alpha=0.1, gamma=0.9,epsilon=0.1,seed=None):
        # 让 Sarsa 自动执行它继承的父类,可以自动执行父类 __init__() 中的所有初始化逻辑。
        super().__init__(env, alpha, gamma, epsilon, seed)
        
    def update_Q_table(self, state, action, reward, next_state, next_action):
        td_target = reward + self.gamma * self.Q_table[next_state, next_action] # 计算 TD 目标
        td_error = td_target - self.Q_table[state, action]
        self.Q_table[state, action] += self.alpha * td_error
        self.policy_is_updated = False # 更新 Q 表后，策略需要更新

#### 二、Expected SARSA:
1. **Expected SARSA 的更新公式：**
    $$
    Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r + \gamma \mathbb{E}_{a_{t+1} \sim \pi(\cdot|s_{t+1})} \left[ Q(s_{t+1}, a_{t+1}) \right] - Q(s_t, a_t) \right]
    $$
    在 ε-greedy 策略下，动作概率为：贪婪动作以概率 $ 1 - \epsilon + \frac{\epsilon}{|\mathcal{A}|} $ 被选择，其他动作以 $ \frac{\epsilon}{|\mathcal{A}|} $ 被选择。将其代入期望项可得：
    $$
    \mathbb{E}_{a' \sim \pi} \left[ Q(s', a') \right] =
    \epsilon \cdot \frac{1}{|\mathcal{A}|} \sum_{a'} Q(s', a') +
    (1 - \epsilon) \cdot \max_{a'} Q(s', a')
    $$
    因此，TD 目标为：
    $$
    Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ r + \gamma \left( \epsilon \cdot \frac{1}{|\mathcal{A}|} \sum_{a'} Q(s_{t+1}, a') + (1 - \epsilon) \cdot \max_{a'} Q(s_{t+1}, a') \right) - Q(s_t, a_t) \right]
    $$
2. **Expected SARSA** 每次给出的 TD target 都是无偏的，因此每一步更新的移动方向都会确定性地减小 TD error，这样下一步更新的移动距离也会相应减小，直到最后 TD error = 0 时，再优化的移动距离也减小到 0，就好像实现了一种自适应学习率的梯度下降优化，所以其学习率可以设置为1。
3. **SARSA 依赖行为策略，不能 Off-policy 使用**  
   SARSA 的 TD 目标使用的是行为策略 $ \mu $ 采样得到的动作 $ a' $，即：
   $$
   \text{TD-target}_{\text{SARSA}} = r + \gamma Q_\mu(s', \mu(s'))
   $$
   所以更新必须依赖生成数据的策略，不能复用旧数据。
4. **Expected SARSA 只依赖目标策略，可 Off-policy 使用**  
   Expected SARSA 的 TD 目标是：
   $$
   \text{TD-target}_{\text{Expected}} = r + \gamma \mathbb{E}_{a' \sim \pi(s')} [Q(s', a')]
   $$
   不依赖行为策略，可直接使用经验回放等旧样本实现 Off-policy 学习。
5. **Q-learning 是 Expected SARSA 的特例**  
   若目标策略$ \pi $ 是 greedy（贪婪）策略，则：
   $$
   \mathbb{E}_{a' \sim \pi(s')} [Q(s', a')] = \max_{a'} Q(s', a')
   $$
   即 Q-learning = Expected SARSA + 贪婪策略。


In [ ]:
class ExpectedSarsa(Solver):
    def __init__(self, env:gym.Env, alpha=0.1, gamma=0.9,epsilon=0.1,seed=None):
        # 让 Sarsa 自动执行它继承的父类,可以自动执行父类 __init__() 中的所有初始化逻辑。
        super().__init__(env, alpha, gamma, epsilon, seed)
        
    def update_Q_table(self, state, action, reward, next_state, batch_size=0):
        # batch_size = 0为on-policy,否则为off-policy
        if batch_size == 0: 
            Q_Exp = (1-self.epsilon) * self.Q_table[next_state].max() + self.epsilon * self.Q_table[next_state].mean()
            td_target = reward + self.gamma * Q_Exp # 计算 TD 目标
            td_error = td_target - self.Q_table[state, action]
            self.Q_table[state, action] += self.alpha * td_error
        else:
            self.replay_buffer.push_transition(transition=(state, action, reward, next_state))
            transitions = self.replay_buffer.sample(batch_size)
            for s,a,r,s_ in transitions:
                Q_Exp = (1-self.epsilon) * self.Q_table[s_].max() + self.epsilon * self.Q_table[s_].mean()
                td_target = r + self.gamma * Q_Exp # 计算 TD 目标
                td_error = td_target - self.Q_table[s, a]
                self.Q_table[s, a] += self.alpha * td_error
        self.policy_is_updated = False # 更新 Q 表后，策略需要更新

#### 三、N-step SARSA:

1. **N-step SARSA 的更新公式：**  
   $$
   Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ G_{t:t+n} - Q(s_t, a_t) \right]
   $$
   其中多步回报 \( G_{t:t+n} \) 定义为：  
   $$
   G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^{n} Q(s_{t+n}, a_{t+n})
   $$
2. 与单步 TD（1-step SARSA）相比，N-step SARSA 利用更多即时奖励，降低了偏差，但增加了估计的方差。
3. 当展开步数 $n \to \infty$ 时，N-step SARSA 退化为蒙特卡洛（Monte Carlo）方法，使用完整轨迹的累积回报：
   $$
   G_t = \sum_{k=t+1}^T \gamma^{k - t - 1} R_k
   $$
   这种方法无偏，但估计方差较大，且不依赖当前价值函数的估计。
4. N-step SARSA 的回报估计在步数较小时，兼具蒙特卡洛方法的低方差和单步 TD 的低偏差优势。 N-step SARSA 可以与 Expected SARSA 结合，形成 N-step Expected SARSA，进一步降低方差，提高更新的稳定性和效率。
5. 标准 N-step SARSA 是 On-policy 方法，依赖行为策略采样数据，因此更新必须跟数据生成策略一致.
7. 通过引入重要性采样，N-step SARSA 能扩展为 Off-policy 方法，从非目标策略生成的轨迹中修正偏差，实现目标策略的无偏学习。**
**重要性采样的数学定义：**  
   在状态 $ s $下动作 $ a $ 的重要性采样权重为：  
   $$
   w(a|s) = \frac{\pi(a|s)}{\mu(a|s)}
   $$
   其中 $ \pi $ 是目标策略，$ \mu $ 是行为策略。
   利用行为策略采样，目标策略期望值可写为：  
   $$
   \mathbb{E}_{a \sim \pi(\cdot|s)}[f(a)] = \sum_a \pi(a|s) f(a) = \sum_a \mu(a|s) w(a|s) f(a) = \mathbb{E}_{a \sim \mu(\cdot|s)}[w(a|s) f(a)]
   $$
 多步重要性采样比率定义为轨迹区间 $ t $ 到 $ h $ 重要性采样权重的乘积： 
    $$
    \rho_{t:h} = \prod_{k=t}^h \frac{\pi(A_k|S_k)}{\mu(A_k|S_k)}
    $$

多步 TD 目标的 Off-policy 更新公式为： 
    $$
    Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \rho_{t:h} \left[ G_{t:h} - Q(S_t, A_t) \right]
    $$
    其中  
    $$
    G_{t:h} = \sum_{i=t+1}^h \gamma^{i-t-1} R_i + \gamma^{h-t} Q(S_h, A_h)
    $$

 **重要性采样的优势:**
 保证 Off-policy 学习的无偏性，但其乘积形式可能导致权重方差极大，影响训练稳定。


In [ ]:
class Nstep_SARSA(Solver):
    """on-policy"""
    def __init__(self, env, alpha=0.1, gamma=0.9, epsilon=0.1, seed=None, nstep=5):
        super().__init__(env, alpha, gamma, epsilon, seed)
        self.nstep = nstep
        self.state_list = []
        self.action_list = []
        self.reward_list = []
    
    def update_Q_table(self, state, action, reward, next_state, next_action, done):
        # 追加当前状态、动作奖励对，注意不用np.append()，因为其每次都会返回一个新的数组，花销大
        self.state_list.append(state)
        self.action_list.append(action)
        self.reward_list.append(reward)
        # 判断是否存储了nstep组数据，N-step SARSA 设计的本质就是基于“完整的n步回报”来更新价值函数。
        if len(self.state_list) == self.nstep:
            # 计算G_{t:t+n}，采用倒序的方法
            G = self.Q_table[next_state,next_action] 
            for i in reversed(range(self.nstep)):
                G = self.gamma * G + self.reward_list[i]       
            td_target = G # TD-target
            # 提取最初的状态动作对，并剔除最老的一组数据
            s_t = self.state_list.pop(0)
            a_t = self.action_list.pop(0)
            self.reward_list.pop(0)
            td_error = td_target - self.Q_table[s_t, a_t]
            self.Q_table[s_t, a_t] = self.Q_table[s_t, a_t] + self.alpha * td_error
        # 如果当智能体进入悬崖或者到达终点，此时大概率不满足长度为nstep,需要特殊处理
        if done:
            # 终止状态特殊处理说明：
            # 当episode提前终止时（如掉下悬崖或到达终点），缓冲区中可能仍有未处理的(s,a,r)序列。
            # 这些序列由于不足n_step长度，无法通过常规n_step更新处理，但依然包含有价值的经验信息。
            # 计算终止状态的回报（G）：
            # 1. 以终止状态的Q值作为初始回报基准（通常Q(s_terminated,a)=0）
            # 2. 反向遍历缓冲区，逐步累积折扣奖励（符合Bellman方程的时间顺序）
            # 计算G_{t:t+m}，采用倒序的方法
            G = self.Q_table[next_state,next_action] 
            for i in reversed(range(len(self.state_list))):
                G = self.gamma * G + self.reward_list[i]       
                
                td_target = G # TD-target
                s_t = self.state_list[i]
                a_t = self.action_list[i]
                td_error = td_target - self.Q_table[s_t, a_t]
                self.Q_table[s_t, a_t] = self.Q_table[s_t, a_t] + self.alpha * td_error
            # 清空列表，开始新的一轮episode
            self.state_list = []
            self.action_list = []
            self.reward_list = []
        self.policy_is_updated = False
            

##### 补充说明：
<a id="算法背景"></a>
1. 算法背景
N-step SARSA是时序差分(TD)学习的重要变体，它：
- 结合了MC方法和单步TD方法的优点
- 通过n步引导(boostrapping)平衡偏差和方差
- 特别适合处理episode长度不固定的场景

<a id="模拟场景设定"></a>
2. 模拟场景设定
2.1 参数配置表
| 参数 | 值 | 说明 |
|------|----|------|
| `n_step` | 3 | 多步更新的步数 |
| `gamma` | 0.9 | 折扣因子 |
| `alpha` | 0.1 | 学习率 |
| 环境 | CliffWalking | 4x12网格世界 |
2.2 初始状态
```python
经验回放缓冲区
state_buffer = [s0, s1]  # 已存储的状态序列
action_buffer = [a0, a1]  # 已执行的动作序列
reward_buffer = [-1, -1]  # 已获得的奖励序列
Q表初始值
Q_table = {
    (s0,a0): 0,  # 初始Q值
    (s1,a1): 0,
    (s2,a2): 0,
    (s3,a3): 0   # 终止状态
}
```
2.3 最新交互事件
Agent在状态`s2`执行动作`a2`后：
```mermaid
graph LR
    A[执行a2] --> B[获得r=-100<br>(掉悬崖)]
    B --> C[到达s3(终止)]
    C --> D[done=True]
```
<a id="完整处理流程"></a>
3. 完整处理流程
3.1 数据准备阶段
```python
将最新transition加入缓冲区
state_buffer.append(s2)    # [s0, s1, s2]
action_buffer.append(a2)   # [a0, a1, a2]
reward_buffer.append(-100) # [-1, -1, -100]
```
3.2 终止状态处理
```python
G = Q_table[s3,a3]  # 终止状态Q值初始化为0
```
迭代更新过程
| 迭代次数 | 计算过程 | 更新操作 | 缓冲区变化 |
|----------|---------|----------|------------|
| 第一次(i=2) | `G = 0.9*0 + (-100) = -100` | 更新(s0,a0):<br>`Q = 0 + 0.1*(-100-0) = -10` | 移除(s0,a0,-1) |
| 第二次(i=1) | `G = 0.9*(-100) + (-1) = -91` | 更新(s1,a1):<br>`Q = 0 + 0.1*(-91-0) = -9.1` | 移除(s1,a1,-1) |
| 第三次(i=0) | `G = 0.9*(-91) + (-100) = -181.9` | 更新(s2,a2):<br>`Q = 0 + 0.1*(-181.9-0) = -18.19` | 移除(s2,a2,-100) |

3.3 最终结果
更新后的Q表
| 状态-动作对 | 更新公式 | 新Q值 |
|-------------|---------|-------|
| (s0,a0) | 0 + 0.1 × (-100) | -10.0 |
| (s1,a1) | 0 + 0.1 × (-91) | -9.1 |
| (s2,a2) | 0 + 0.1 × (-181.9) | -18.19 |

缓冲区状态
```python
assert len(state_buffer) == 0  # 已完全清空
```
<a id="关键机制解析"></a>
4. 关键机制解析
4.1 反向更新原理
```mermaid
graph RL
    A[终止Q=0] -->|γ×0 + r2| B[G=-100]
    B -->|更新(s2,a2)| C
    C -->|γ×(-100) + r1| D[G=-91]
    D -->|更新(s1,a1)| E
    E -->|γ×(-91) + r0| F[G=-181.9]
    F -->|更新(s0,a0)| G
```
4.2 数学本质
采用修正的n步回报：
$$
G_{t:t+n} = \sum_{k=0}^{n-1} \gamma^k R_{t+k+1} + \gamma^n Q(S_{t+n}, A_{t+n})
$$
当遇到终止状态时：
$$
\gamma^n Q(S_{t+n}, A_{t+n}) = 0
$$



#### 四、N-step SARSA (Off-policy)

##### 1. 算法概述

相比于标准的 On-policy N-step SARSA，Off-policy 版本通过行为策略（behavior policy）和目标策略（target policy）分离，允许使用不同的策略进行探索和学习，从而提高样本效率。
- **核心思想**：
  - 使用行为策略 $\mu$（如 $\epsilon$-greedy）采样动作，生成经验轨迹。
  - 使用目标策略 $\pi$（如贪婪策略）评估和优化 Q 值。
  - 通过重要性采样（Importance Sampling）修正行为策略与目标策略之间的偏差。
  - 利用 n 步回报更新 Q 值，平衡单步 TD 和蒙特卡洛方法的优缺点。
- **适用场景**：
  - 需要高效利用经验数据的场景。
  - 希望通过探索性强的行为策略收集数据，同时优化目标策略的场景。

---

##### 2. 算法原理

###### 2.1 N-step SARSA 基础

N-step SARSA 是单步 SARSA 的扩展，使用 n 步回报来更新 Q 值。基本更新公式为：
$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[ G_{t:t+n} - Q(s_t, a_t) \right]
$$
其中：
- $G_{t:t+n}$ 是 n 步回报，定义为：
$$
G_{t:t+n} = \sum_{k=t+1}^{\min(t+n, T)} \gamma^{k-t-1} r_k + \gamma^n Q(s_{t+n}, a_{t+n})
$$
- $T$ 是终止时刻，$\gamma$ 是折扣因子，$\alpha$ 是学习率。
- $a_{t+n}$ 是根据当前策略采样的动作（On-policy 场景下）。

###### 2.2 Off-policy 扩展
在 Off-policy 场景中，行为策略 $\mu$ 和目标策略 $\pi$ 不同，直接使用$\mu$ 采样的 $G_{t:t+n}$ 会导致偏差。为此，引入重要性采样比率 $\rho$ 修正回报估计：
$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \rho_{t:t+n-1} \left[ G_{t:t+n} - Q(s_t, a_t) \right]
$$
其中：
- $\rho_{t:t+n-1} = \prod_{k=t}^{t+n-1} \frac{\pi(a_k|s_k)}{\mu(a_k|s_k)}$，表示 n 步动作序列的概率比。
- $\pi(a|s)$ 是目标策略的概率，$\mu(a|s)$ 是行为策略的概率。

###### 2.3 重要性采样的作用
重要性采样确保 Off-policy 估计的无偏性：
$$
\mathbb{E}_{\mu} \left[ \rho_{t:t+n-1} G_{t:t+n} \right] = \mathbb{E}_{\pi} \left[ G_{t:t+n} \right]
$$
但 $\rho$ 的乘积可能导致方差较大，尤其当 $\mu$ 和 $\pi$ 差异较大时。

---

##### 3. 数学推导

###### 3.1 n 步回报的计算

从 $t$ 到 $t+n$ 的回报 $G_{t:t+n}$ 包含两部分：
1. 前 $n$ 步的实际奖励：$\sum_{k=t+1}^{t+n} \gamma^{k-t-1} r_k$。
2. 第 $n$ 步的估计值：$\gamma^n Q(s_{t+n}, a_{t+n})$。

如果 episode 在 $t+n$ 之前终止（即 $T < t+n$），则 $G_{t:t+n} = \sum_{k=t+1}^{T} \gamma^{k-t-1} r_k$，且后续 Q 值为 0。

###### 3.2 重要性采样比率
对于每步动作 \(a_k\)，重要性采样比率为：
$$
\rho_k = \frac{\pi(a_k|s_k)}{\mu(a_k|s_k)}
$$
- 假设目标策略$\pi$ 是贪婪策略：$\pi(a|s) = 1/|\mathcal{A}_{\text{greedy}}|$（若 $a$ 是贪婪动作，否则为 0）。
- 行为策略 $\mu$ 是 $\epsilon$-greedy：$\mu(a|s) = (1-\epsilon)/|\mathcal{A}_{\text{greedy}}| + \epsilon/|\mathcal{A}|$。
- 总比率 $\rho_{t:t+n-1} = \prod_{k=t}^{t+n-1} \rho_k$。

###### 3.3 更新公式推导
目标是估计目标策略 $\pi$ 下的 Q 值：
$$
Q_{\pi}(s_t, a_t) = \mathbb{E}_{\pi} \left[ G_{t:t+n} \right]
$$
使用行为策略 $\mu$ 采样数据后，通过重要性采样修正：
$$
\mathbb{E}_{\mu} \left[ \rho_{t:t+n-1} G_{t:t+n} \right] = \mathbb{E}_{\pi} \left[ G_{t:t+n} \right]
$$
因此，更新公式为：
$$
Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \rho_{t:t+n-1} \left[ G_{t:t+n} - Q(s_t, a_t) \right]
$$

---

In [ ]:
class NstepSarsa_Off_policy(Solver):
    def __init__(self, env, alpha=0.1, gamma=0.9, epsilon=0.1, seed=None, nstep=5):
        super().__init__(env, alpha, gamma, epsilon, seed)
        self.nstep = nstep
        self.state_list = []
        self.action_list = []
        self.reward_list = []
        # 保存每一步的重要性采样
        self.rho_list = []
    
    def update_Q_table(self, state, action, reward, next_state, next_action, done):
        self.state_list.append(state)
        self.action_list.append(action)
        self.reward_list.append(reward)
        
        # 判定该动作是否为ε-greedy的策略
        if action in self.greedy_policy[state]:
            p_target = 1 / self.greedy_policy[state].shape[0] # 目标策略
            p_behavior = (1 - self.epsilon) / self.greedy_policy[state].shape[0] + self.epsilon / self.n_action # 行为策略
            self.rho_list.append(p_target / p_behavior)
        else:
            self.rho_list.append(0)
        # 套用 NstepSarsa
        # 判断是否存储了nstep组数据，N-step SARSA 设计的本质就是基于“完整的n步回报”来更新价值函数。
        if len(self.state_list) == self.nstep:
            # 计算G_{t:t+n}，采用倒序的方法
            G = self.Q_table[next_state,next_action] 
            rho = 1
            for i in reversed(range(self.nstep)):
                G = self.gamma * G + self.reward_list[i]       
                rho *= self.rho_list[i]
                
            td_target = G # TD-target
            # 提取最初的状态动作对，并剔除最老的一组数据
            s_t = self.state_list.pop(0)
            a_t = self.action_list.pop(0)
            self.reward_list.pop(0)
            td_error = td_target - self.Q_table[s_t, a_t]
            self.Q_table[s_t, a_t] = self.Q_table[s_t, a_t] + self.alpha * rho * td_error
        # 如果当智能体进入悬崖或者到达终点，此时大概率不满足长度为nstep,需要特殊处理
        if done:
            # 终止状态特殊处理
            G = self.Q_table[next_state,next_action] 
            rho = 1
            for i in reversed(range(len(self.state_list))):
                G = self.gamma * G + self.reward_list[i]    
                rho *= self.rho_list[i]
                   
                td_target = G # TD-target
                # 提取最初的状态动作对，并剔除最老的一组数据
                s_t = self.state_list[i]
                a_t = self.action_list[i]
                td_error = td_target - self.Q_table[s_t, a_t]
                self.Q_table[s_t, a_t] = self.Q_table[s_t, a_t] + self.alpha * rho * td_error
            # 清空列表，开始新的一轮episode
            self.state_list = []
            self.action_list = []
            self.reward_list = []
        self.policy_is_updated = False

#### 两种 $\epsilon$-greedy 策略总结对比

##### 1. 普通的 $\epsilon$-greedy 策略（Single Optimal Action）

- **定义**
- 以概率 $1-\epsilon$ 选择当前状态下 **Q 值最大的一个动作**（单一最优动作）。
- 以概率 $\epsilon$ 均匀随机选择一个动作（包括最优动作和非最优动作）。

- **数学表达**
假设动作空间大小为 $ |\mathcal{A}| $ ，唯一的最优动作为 $a^*$，则动作 $a$ 被选中的概率为：

$$
\mu(a|s) = \begin{cases}
1 - \epsilon + \frac{\epsilon}{|\mathcal{A}|}, & a = a^* \\
\frac{\epsilon}{|\mathcal{A}|}, & a \neq a^*
\end{cases}
$$

- **特点**
- 只考虑一个最大 Q 值动作，若多个最大值默认选第一个或随机选一个。
- 实现简单，广泛应用于 Q-learning、SARSA 等。
- 随 $\epsilon$ 变化，从探索转向利用。

---

##### 2. 多最优动作均匀分布的 $\epsilon$-greedy 策略（Multiple Optimal Actions）

- **定义**
- 假设存在 $k$ 个动作共享最大 Q 值（即多个最优动作）。
- 以概率 $1-\epsilon$ 在这 $k$ 个最优动作中均匀选择一个动作。
- 以概率 $\epsilon$ 在整个动作空间 $|\mathcal{A}|$ 中均匀随机选择一个动作。

- **数学表达**
动作 $a$ 被选中的概率为：

$$
\mu(a|s) = (1-\epsilon) \cdot \frac{\mathbb{I}[a \in \text{optimal}]}{k} + \epsilon \cdot \frac{1}{|\mathcal{A}|}
$$

其中，$\mathbb{I}[a \in \text{optimal}]$ 表示动作 $a$ 是否属于最优动作集合。

- **特点**
- 公平分配贪婪选择概率给多个最优动作。
- 适用于多个最优动作平局的情况。
- 代码实现需先统计所有最优动作数量 $k$。

---

##### 3. 二者对比表

| 特征           | 普通 $\epsilon$-greedy                 | 多最优动作 $\epsilon$-greedy               |
|----------------|-----------------------------------------|---------------------------------------------|
| 最优动作数量   | 单一（选一个最大 Q 动作）                | 多个（所有最大 Q 动作均匀分布）             |
| 贪婪选择概率分布 | 全部给单一动作                           | 均分给所有最优动作                           |
| 随机探索       | 均匀随机所有动作                         | 同左                                         |
| 计算复杂度     | 低                                      | 需统计多个最优动作                           |
| 适用环境       | 简单环境，单一最优动作                   | 多最优动作平局场景                           |
| 代码实现难度   | 简单                                    | 稍复杂，需要统计最优动作集合                  |

---


#### 五、Q-Learning:
##### 1.定义
Q-Learning 是一种 **Off-policy** 的无模型强化学习算法，目标是学习一个最优动作-价值函数 $ Q^*(s, a) $，表示在状态 $ s $ 下选择动作 $ a $ 的期望累积回报（折扣回报）。它通过与环境交互逐步更新 Q 值，最终收敛到最优策略 $\pi^*$。
- **Off-policy**：Q-Learning 的行为策略（用于探索）和目标策略（用于优化）是分离的。行为策略通常是 $\epsilon$-greedy，而目标策略是贪婪策略（选择最大 Q 值的动作）。
- **无模型**：不需要知道环境的转移概率或奖励函数，直接从经验中学习。
- 适合离散状态和动作空间的问题，如 Cliff Walking 环境。
- 用于机器人导航、游戏 AI 等需要动态决策的场景。
---

##### 2. Q-Learning 的原理

###### 2.1 核心思想
Q-Learning 的目标是逼近最优 Q 函数 $ Q^*(s, a) $，满足贝尔曼最优方程：
$$
Q^*(s, a) = \mathbb{E} \left[ R + \gamma \max_{a'} Q^*(s', a') \mid s, a \right]
$$
- $ R $：即时奖励。
- $\gamma$：折扣因子（0 ≤ $\gamma$ < 1），表示未来回报的重要性。
- $ s' $：执行动作 $ a $ 后的下一状态。
- $\max_{a'} Q^*(s', a')$：下一状态下最优动作的 Q 值。

Q-Learning 通过迭代更新 Q 值，使其逐步接近 $ Q^* $。

###### 2.2 更新公式
根据 PDF 中的公式，Q-Learning 的更新规则为：
$$
Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]
$$
- **$ Q(s, a) $**：当前状态-动作对的 Q 值估计。
- **$ \alpha $**：学习率（0 < $\alpha$ ≤ 1），控制更新步长。
- **$ r $**：从状态 $ s $执行动作 $ a $ 获得的即时奖励。
- **$ \gamma \max_{a'} Q(s', a') $**：折扣后的最优未来回报，$\max_{a'}$ 表示选择下一状态 \( s' \) 下 Q 值最大的动作。
- **$ Q(s, a) $ 的 TD 误差**：$ r + \gamma \max_{a'} Q(s', a') - Q(s, a) $是时间差分（TD）误差，用于调整 Q 值。
- 随着 \(\alpha \to 0\) 且 \(\sum \alpha = \infty\)，Q-Learning 收敛到 \( Q^* \)（根据 Stochastic Approximation 理论）。

###### 2.3 与 SARSA 的区别
- **Q-Learning**：
  - Off-policy，目标使用 $\max_{a'} Q(s', a')$，假设总是选择最优动作。
  - 不依赖行为策略的下一步动作 \( a' \)。
- **SARSA**：
  - On-policy，目标使用 $ Q(s', a') $，其中 $ a' $ 是根据当前策略（例如 $\epsilon$-greedy）采样的动作。
  - 依赖行为策略的实际采样。

在 Cliff Walking 中，Q-Learning 倾向于快速学习最优路径，而 SARSA 可能更保守，考虑探索的影响。

---

##### 6. 优点与缺点

###### 6.1 优点
- **Off-policy**：可以独立于行为策略学习最优策略，适合多种探索策略。
- **简单高效**：无需环境模型，计算量主要来自 Q 表更新。
- **收敛性**：在离散空间中，理论上能收敛到最优解。

###### 6.2 缺点
- **计算开销**：Q 表存储需求随状态和动作数量增长呈指数级增加（维度诅咒）。
- **过估计问题**：$\max_{a'} Q(s', a')$ 可能高估 Q 值，导致不稳定（Double Q-Learning 可缓解）。
- **探索依赖**：$\epsilon$ 设置不当可能导致探索不足或过度。

---

In [ ]:
class Q_learning(Solver):
    def __init__(self, env:gym.Env, alpha=0.1, gamma=0.9,epsilon=0.1,seed=None):
        super().__init__(env, alpha, gamma, epsilon, seed)
        
    def update_Q_table(self, state, action, reward, next_state, batch_size=0):
        if batch_size == 0: # on-policy
            td_target = reward + self.gamma * self.Q_table[next_state].max()
            td_error = td_target - self.Q_table[state, action]
            self.Q_table[state, action] += self.alpha * td_error
        else: # off-policy
            self.replay_buffer.push_transition(transition=(state, action, reward, next_state))
            transitions = self.replay_buffer.sample(batch_size)
            for s, a, r, s_ in transitions:
                td_target = r + self.gamma * self.Q_table[s_].max()
                td_error = td_target - self.Q_table[s, a]
                self.Q_table[s,a] += self.alpha * td_error 
        self.policy_is_updated = False
            